In [ ]:
!pip install sentencepiece transformers

In [ ]:
MODEL_SAVE_PATH = 'MODEL_SAVE_PATH'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_SAVE_PATH).cuda();
tokenizer = NllbTokenizer.from_pretrained(MODEL_SAVE_PATH)

In [ ]:
def translate(
    text, src_lang='rus_Cyrl', tgt_lang='eng_Latn',
    a=32, b=3, max_input_length=1024, num_beams=4, **kwargs
):
    """Turn a text or a list of texts into a list of translations"""
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(
        text, return_tensors='pt', padding=True, truncation=True,
        max_length=max_input_length
    )
    model.eval() # turn off training mode
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams, **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [ ]:
# Example usage:
t = 'А̄лы ма̄т о̄лнэ то̄ва ӯйхулт вит та̄нти палтыланыл ӯньщим о̄луӈкв ханьщувласт,  таи ма̄гсыл хо̄са вит тал о̄луӈкв вēрмегы.'
print(translate(t, 'mns_Cyrl', 'rus_Cyrl'))
# ['Обитатели южных районов научились накапливать в организме запасы влаги, по этому по долгу могут обходиться без воды.']

In [ ]:
# Example usage:
t = 'Его много народу знает, во многих странах известен.'
print(translate(t, 'rus_Cyrl', 'mns_Cyrl'))
# ['Тав са̄в хо̄тпан ва̄ве, са̄в ма̄т ва̄ве.']